In [6]:
:dep plotters = { version = "^0.3.0", default_features = false, features = ["evcxr", "all_series", "all_elements", "chrono"] }
:dep rstrading = { version = "*", path="../" }
:dep csv = { version = "*" }
:dep serde = { version = "1.0.143", features = ["derive"] }
:dep chrono = { version = "0.4.26" }

extern crate plotters;
use plotters::prelude::*;
use plotters::coord::ranged1d::*;
use plotters::coord::Shift;

extern crate rstrading;
use rstrading::data::Candle;
use rstrading::indicators::*;
use crate::rstrading::Next;

extern crate csv;
use std::fs::File;
use serde::{Deserialize};


use chrono::{DateTime, TimeZone, NaiveDateTime, Utc};

In [33]:
/// Load the json data into variables

const DATA_HISTORIC_XAUUSD_CANDLES_1M: &str = "./candles/xauusd_1m.csv";

#[derive(Deserialize)]
struct Record {
    ticker: String,
    date: String,
    time: String,
    open: f64,
    high: f64,
    low: f64,
    close: f64,
    vol: i32,
}

fn load_json_market_event_candles() -> (Vec<Candle>, DateTime<Utc>, DateTime<Utc>, f64, f64) {
    let file = File::open(DATA_HISTORIC_XAUUSD_CANDLES_1M).unwrap();
    let mut rdr = csv::ReaderBuilder::new().has_headers(true).from_reader(file);
    let mut candles = Vec::new();
    
    let mut xMin = chrono::DateTime::<Utc>::MAX_UTC;
    let mut xMax = chrono::DateTime::<Utc>::MIN_UTC;
    
    let mut yMin = f64::MAX;
    let mut yMax = f64::MIN;
    
    for result in rdr.deserialize::<Record>() {
        let record = result.unwrap();
        let date_str = record.date + " " + record.time.as_str() + " +00:00";
        let close_time = DateTime::parse_from_str(&date_str, "%Y%m%d %H%M%S %z").unwrap().with_timezone(&Utc);
        //let close_time_millis = close_time;
        candles.push(Candle{
            close_time,
            open: record.open,
            high: record.high,
            low: record.low,
            close: record.close,
            volume: Some(record.vol as f64),
            trade_count: None,
        });
        
        if (close_time > xMax) {
            xMax = close_time;
        }
        
        if (close_time < xMin) {
            xMin = close_time;
        }
        
        if (record.high > yMax) {
            yMax = record.high;
        }
        
        if (record.low < yMin) {
            yMin = record.low;
        }
    }
    
    (candles, xMin, xMax, yMin, yMax)
}

let (candles, xMin, xMax, yMin, yMax) = load_json_market_event_candles();
println!("{:?}\n", &candles[0..3]);

[Candle { close_time: 2022-06-08T00:00:00Z, open: 1849.0, high: 1849.1, low: 1849.0, close: 1849.0, volume: Some(4.0), trade_count: None }, Candle { close_time: 2022-06-08T00:01:00Z, open: 1849.1, high: 1849.3, low: 1849.0, close: 1849.3, volume: Some(4.0), trade_count: None }, Candle { close_time: 2022-06-08T00:02:00Z, open: 1849.4, high: 1849.4, low: 1849.2, close: 1849.3, volume: Some(4.0), trade_count: None }]



In [34]:
// Calculate ema
fn calculate_ema(candles: &Vec<Candle>, duration: usize) -> Vec<(DateTime<Utc>, f64)> {
    let mut ema = ExponentialMovingAverage::new(duration).unwrap();

    let mut ema_line_data: Vec<(DateTime<Utc>, f64)> = Vec::new();
    for i in 0..candles.len() {
        ema_line_data.push((candles[i].close_time, ema.next(candles[i].open)));
    }
    ema_line_data
}

let ema_line_data = calculate_ema(&candles, 15);

println!("{:?}\n", &ema_line_data[0..3]);

[(2022-06-08T00:00:00Z, 1849.0), (2022-06-08T00:01:00Z, 1849.0125), (2022-06-08T00:02:00Z, 1849.0609375)]



In [37]:
// Draw the chart
let chart_width = 1280;
evcxr_figure((chart_width, 400), |root| {
    root.fill(&WHITE)?;
    let mut chart = ChartBuilder::on(&root)
        .x_label_area_size(60)
        .y_label_area_size(60)
        .build_cartesian_2d(xMin..xMax, yMin..yMax)
        .unwrap();
    
    chart
        .configure_mesh()
        .light_line_style(&WHITE)
        .draw()
        .unwrap();
    
    let candle_width = (chart_width as usize) / candles.len();
    chart.draw_series(candles.iter().map(|x| {
        CandleStick::new(
            x.close_time,
            x.open,
            x.high,
            x.low,
            x.close,
            &GREEN,
            &RED,
            candle_width as u32,
        )
    })).unwrap();
    
    chart.draw_series(LineSeries::new(ema_line_data.clone(), BLUE.stroke_width(1))).unwrap();  

    Ok(())
})

1846.0
 
 
 
1847.0
 
 
 
1848.0
 
 
 
1849.0
 
 
 
1850.0
 
 
 
1851.0
 
 
 
 
2022-06-08T00:00:00Z
 
 
 
2022-06-08T01:00:00Z
 
 
 
2022-06-08T02:00:00Z
 
 
 
2022-06-08T03:00:00Z
 
 
 
2022-06-08T04:00:00Z
 
 
 
2022-06-08T05:00:00Z
 
 
 
2022-06-08T06:00:00Z
 
 
 
2022-06-08T07:00:00Z
 
 
 
2022-06-08T08:00:00Z
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline fill="none" opacity="1" stroke="#0000FF" stroke-width="1" points="60,128 62,127 64,124 67,123 69,118 72,113 74,109 77,105 79,101 82,95 84,95 86,97 89,98 91,99 94,100 96,102 99,104 101,106 104,109 106,111 108,110 111,109 113,107 116,106 118,106 121,106 123,105 126,102 128,100 130,100 133,103 135,104 138,105 140,108 143,108 145,108 148,109 150,106 153,105 155,102 157,98 160,95 162,96 165,96 167,97 170,97 172,95 175,95 177,95 179,93 182,93 184,94 187,96 189,97 192,99 194,101 197,105 199,108 201,111 204,115 206,118 209,122 211,125 214,128 216,132 219,135 221,139 224,141 226,143 228,146 231,150 233,156 236,162 238,166 241,171 243,179 246,184 248,187 250,188 253,190 255,19